# Pynq2
Pynq2 shall hold an RGB LED and Pmod AD2 + buzzer
Pynq2 shall be a server waiting for connection
Pynq2 shall, upon collection of ADC data shall determine whether to beep or blink the sensors

In [1]:
# Buzzer
#from pynq.lib import Pmod_ADC
#adc = Pmod_ADC(base.PMODB)

# RGB
#from pynq.overlays.base import BaseOverlay
#baseA = BaseOverlay("base.bit")
#baseB = BaseOverlay("base.bit")

# Multiprocessing
import multiprocessing as mp
from multiprocessing import Process, Value, Array
import os
from time import sleep
import time 
# Networking
import socket
import queue

from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"
//#include "stdio.h"

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    //pyprintf("write to gpio : pin% value%",pin,val);
    //#print("write called");
    //printf("write called");
    read_gpio(pin);
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    // gpio_close(pin_out);
}


//1. Code to reset all pins
void reset_all_gpio_pins() {
    pyprintf("reset_all_gpio_pins");
     for(int i=0;i<8;i++) {
        write_gpio(i,0);
    }
}

https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units

In [3]:
from scipy.signal import butter, lfilter

def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Filter requirements.
order = 6
fs = 10       # sample rate, Hz
cutoff = 3.0  # desired cutoff frequency of the filter, Hz
rgb_buzzer_freq =1  # X beats/(1minute*60seconds) Hz
rgb_buzzer_period = 1/rgb_buzzer_freq
#last_known_moving_avg_data = 0
#last_known_moving_avg_data_final =0

In [6]:
# For debugging purposes
from IPython.display import display, clear_output
from statistics import mean

#def client_process(host,port):
def client_process(host,port,last_known_moving_avg_data):
    global rgb_buzzer_period
    #global last_known_moving_avg_data, last_known_moving_avg_data_final
    
    raw_data=0
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, port))
    
    data_queue = queue.Queue(100)

    while True:
        data = s.recv(6).decode()
        #clear_output(wait=True)
        data_queue.put(float(data))
        raw_data=mean(list(data_queue.queue))*3000
        #process_input_data(raw_data,last_known_moving_avg_data)
        process_input_data_hb(raw_data,last_known_moving_avg_data)
        print(f"client process : last_known_moving_avg_dat = {float(last_known_moving_avg_data.value)}")
        # Lock Buzzer/RGB
        # Set Buzzer/RGB
        # Unlock Buzzer/RGB
        if data_queue.full():
            data_queue.get()    
    s.close()

def beep_buzzer(duration) :
    #reset_all_gpio_pins();
    buzzer=3
    cnt=0
    print(f"beep_buzzer total beep duration ={float(duration)}")
    #write_gpio(1,0)
    #while cnt<2 :
    #while True :
    if True :
        cnt=cnt+1
        write_gpio(buzzer,0)
        write_gpio(buzzer-1,0)
        time.sleep(duration)
        write_gpio(buzzer,1)
        write_gpio(buzzer-1,1)
        time.sleep(duration)

    #write_gpio(buzzer,0)    
    
    
def process_input_data(input_data,output_data) :
    output_data.value = input_data/(60)   #Divide by 60 get heart rate and then divide by for hi and low time
    #output_data = input_data/(60)   #Divide by 60 get heart rate and then divide by for hi and low time
    print(f"client process : process_input_data = input={input_data} output_data or duration={output_data}")

def process_input_data_hb(input_data,output_data) :
    output_data.value = input_data/(60)   #Divide by 60 get heart rate and then divide by for hi and low time
    #output_data = input_data/(60)   #Divide by 60 get heart rate and then divide by for hi and low time
    print(f"client process : process_input_data_hb = input={input_data} output_data or duration={output_data}")

    
def toggle_buzzer_rgb(input_data):
    output_data =0 
    #global last_known_moving_avg_data
    cnt =0
    while True or cnt<10 : 
        cnt=cnt+1
        #last_known_moving_avg_data_ = input_data
        print(f"toggle_buzzer_rgb processed data for beeping  is ={float(last_known_moving_avg_data.value)}")
        #process_input_data(last_known_moving_avg_data,output_data)
        if(last_known_moving_avg_data.value<0 or last_known_moving_avg_data.value > 5) : 
            print(f"Wrong DATA DONT BEEP = {float(last_known_moving_avg_data.value)}")
            continue
        duration=last_known_moving_avg_data.value
        beep_buzzer(duration)
    
    
    

In [7]:
##if __name__ == '__main__':
last_known_moving_avg_data = mp.Value('f', 0.0)

#client_process = multiprocessing.Process(target=client_process,
#                                         args=('192.168.0.148',5000))


client_process = mp.Process(target=client_process,
                                         args=('192.168.0.148',5001,last_known_moving_avg_data))

#Initiate BUzzer/RGB task
#input_data=0
buzzer_process = mp.Process(target=toggle_buzzer_rgb,args=(last_known_moving_avg_data,))
                                         

client_process.start() 
os.system("taskset -pc {} {}".format(0, client_process.pid)) 


buzzer_process.start() 
os.system("taskset -pc {} {}".format(1, buzzer_process.pid)) 


client_process.join()
print("Client Process " + client_process.name + " is finished")

buzzer_process.join()
#print("Buzzer Process " + buzzer_process + " is finished")
print("buzzer_process finished")

toggle_buzzer_rgb processed data for beeping  is =0.0
beep_buzzer total beep duration =0.6100000143051147client process : process_input_data = input=36.6 output_data or duration=<Synchronized wrapper for c_float(0.6100000143051147)>

client process : last_known_moving_avg_dat = 0.6100000143051147
client process : process_input_data = input=36.6 output_data or duration=<Synchronized wrapper for c_float(0.6100000143051147)>
client process : last_known_moving_avg_dat = 0.6100000143051147
client process : process_input_data = input=36.6 output_data or duration=<Synchronized wrapper for c_float(0.6100000143051147)>
client process : last_known_moving_avg_dat = 0.6100000143051147
client process : process_input_data = input=37.35 output_data or duration=<Synchronized wrapper for c_float(0.6225000023841858)>
client process : last_known_moving_avg_dat = 0.6225000023841858
client process : process_input_data = input=38.1 output_data or duration=<Synchronized wrapper for c_float(0.6349999904632568

client process : process_input_data = input=40.203896103896106 output_data or duration=<Synchronized wrapper for c_float(0.6700649261474609)>
client process : last_known_moving_avg_dat = 0.6700649261474609
client process : process_input_data = input=40.138461538461534 output_data or duration=<Synchronized wrapper for c_float(0.6689743399620056)>
client process : last_known_moving_avg_dat = 0.6689743399620056
client process : process_input_data = input=40.15063291139241 output_data or duration=<Synchronized wrapper for c_float(0.669177234172821)>
client process : last_known_moving_avg_dat = 0.669177234172821
client process : process_input_data = input=40.23375 output_data or duration=<Synchronized wrapper for c_float(0.6705625057220459)>
client process : last_known_moving_avg_dat = 0.6705625057220459
client process : process_input_data = input=40.27777777777778 output_data or duration=<Synchronized wrapper for c_float(0.6712962985038757)>
client process : last_known_moving_avg_dat = 0.6

client process : process_input_data = input=40.272 output_data or duration=<Synchronized wrapper for c_float(0.6711999773979187)>
client process : last_known_moving_avg_dat = 0.6711999773979187
client process : process_input_data = input=40.314 output_data or duration=<Synchronized wrapper for c_float(0.6718999743461609)>
client process : last_known_moving_avg_dat = 0.6718999743461609
client process : process_input_data = input=40.196999999999996 output_data or duration=<Synchronized wrapper for c_float(0.669950008392334)>
client process : last_known_moving_avg_dat = 0.669950008392334
client process : process_input_data = input=40.152 output_data or duration=<Synchronized wrapper for c_float(0.6692000031471252)>
client process : last_known_moving_avg_dat = 0.6692000031471252
client process : process_input_data = input=40.196999999999996 output_data or duration=<Synchronized wrapper for c_float(0.669950008392334)>
client process : last_known_moving_avg_dat = 0.669950008392334
client pro

client process : process_input_data = input=40.671 output_data or duration=<Synchronized wrapper for c_float(0.6778500080108643)>
client process : last_known_moving_avg_dat = 0.6778500080108643
client process : process_input_data = input=40.671 output_data or duration=<Synchronized wrapper for c_float(0.6778500080108643)>
client process : last_known_moving_avg_dat = 0.6778500080108643
client process : process_input_data = input=40.611000000000004 output_data or duration=<Synchronized wrapper for c_float(0.6768500208854675)>
client process : last_known_moving_avg_dat = 0.6768500208854675
client process : process_input_data = input=40.611000000000004 output_data or duration=<Synchronized wrapper for c_float(0.6768500208854675)>
client process : last_known_moving_avg_dat = 0.6768500208854675
client process : process_input_data = input=40.569 output_data or duration=<Synchronized wrapper for c_float(0.6761500239372253)>
client process : last_known_moving_avg_dat = 0.6761500239372253
client

client process : process_input_data = input=41.808 output_data or duration=<Synchronized wrapper for c_float(0.6967999935150146)>
client process : last_known_moving_avg_dat = 0.6967999935150146
client process : process_input_data = input=41.865 output_data or duration=<Synchronized wrapper for c_float(0.6977499723434448)>
client process : last_known_moving_avg_dat = 0.6977499723434448
client process : process_input_data = input=41.937 output_data or duration=<Synchronized wrapper for c_float(0.6989499926567078)>
client process : last_known_moving_avg_dat = 0.6989499926567078
client process : process_input_data = input=41.967 output_data or duration=<Synchronized wrapper for c_float(0.6994500160217285)>
client process : last_known_moving_avg_dat = 0.6994500160217285
client process : process_input_data = input=42.039 output_data or duration=<Synchronized wrapper for c_float(0.7006499767303467)>
client process : last_known_moving_avg_dat = 0.7006499767303467
client process : process_input

...

client process : process_input_data = input=38.289 output_data or duration=<Synchronized wrapper for c_float(0.6381499767303467)>
client process : last_known_moving_avg_dat = 0.6381499767303467
client process : process_input_data = input=38.391 output_data or duration=<Synchronized wrapper for c_float(0.6398500204086304)>
client process : last_known_moving_avg_dat = 0.6398500204086304
client process : process_input_data = input=38.364000000000004 output_data or duration=<Synchronized wrapper for c_float(0.6394000053405762)>
client process : last_known_moving_avg_dat = 0.6394000053405762
client process : process_input_data = input=38.307 output_data or duration=<Synchronized wrapper for c_float(0.638450026512146)>
client process : last_known_moving_avg_dat = 0.638450026512146
client process : process_input_data = input=38.394 output_data or duration=<Synchronized wrapper for c_float(0.6399000287055969)>
client process : last_known_moving_avg_dat = 0.6399000287055969
client process : pro

client process : last_known_moving_avg_dat = 0.6441500186920166
client process : process_input_data = input=38.649 output_data or duration=<Synchronized wrapper for c_float(0.6441500186920166)>
client process : last_known_moving_avg_dat = 0.6441500186920166
client process : process_input_data = input=38.676 output_data or duration=<Synchronized wrapper for c_float(0.644599974155426)>
client process : last_known_moving_avg_dat = 0.644599974155426
client process : process_input_data = input=38.646 output_data or duration=<Synchronized wrapper for c_float(0.64410001039505)>
client process : last_known_moving_avg_dat = 0.64410001039505
client process : process_input_data = input=38.601 output_data or duration=<Synchronized wrapper for c_float(0.6433500051498413)>
client process : last_known_moving_avg_dat = 0.6433500051498413
client process : process_input_data = input=38.646 output_data or duration=<Synchronized wrapper for c_float(0.64410001039505)>
client process : last_known_moving_avg

client process : process_input_data = input=39.396 output_data or duration=<Synchronized wrapper for c_float(0.6565999984741211)>
client process : last_known_moving_avg_dat = 0.6565999984741211
client process : process_input_data = input=39.366 output_data or duration=<Synchronized wrapper for c_float(0.6560999751091003)>
client process : last_known_moving_avg_dat = 0.6560999751091003
client process : process_input_data = input=39.396 output_data or duration=<Synchronized wrapper for c_float(0.6565999984741211)>
client process : last_known_moving_avg_dat = 0.6565999984741211
client process : process_input_data = input=39.411 output_data or duration=<Synchronized wrapper for c_float(0.6568499803543091)>
client process : last_known_moving_avg_dat = 0.6568499803543091
client process : process_input_data = input=39.249 output_data or duration=<Synchronized wrapper for c_float(0.6541500091552734)>
client process : last_known_moving_avg_dat = 0.6541500091552734
toggle_buzzer_rgb processed da

Process Process-1:
Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-44445445ef83>", line 17, in client_process
    data = s.recv(6).decode()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
KeyboardInterrupt
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-44445445ef83>", line 68, in toggle_buzzer_rgb
    beep_buzzer(duration)
  File "<ipython-input-6-44445445ef83>", line 45, in beep_buzzer
    time.sleep(duration)
KeyboardInterrupt


KeyboardInterrupt: 

In [8]:
write_gpio(3,0) 